# Les variables binaires en optimisation 
Tentative de clarification simplifiée, et étude par estelle derrien

<b>CREATION EN COURS </b>

- Valeurs : Elles prennent la valeur 0 ou 1 (non ou oui).
- Intérêt : Sélectionner une variable de décision , pénaliser une variable de décision, sélectionner une contrainte, discriminer une contrainte, établir une condition.
- Utilisation : Directement dans la fonction objectif, ou avec les contraintes.
- Déclaration : Elle diffère selon les solveurs, elle est plus facile dans certains.
- Semblable à : IF/THEN ( Si/Alors en Anglais ), Semblable à un principe ON/OFF pour les machines. Equivaut à un 'trigger' en Anglais, un déclencheur.
- Relaté à : Les méthodes <b>"Big M"</b>.
- Permettent souvent de : Eviter de faire une optimisation non linéaire.
- Permettent d'attribuer une <b>pénalité</b> à une variable de décision, si sa valeur dépasse un certain seuil.
- On évite d'utiliser la fonctionnalité de limites (Bounds) sur les variables de décision souvent fournies avec le solveur, <b>en même temps que</b> les variables binaires, c'est soit l'un,soit l'autre.

# Exemples simples:

Si la production de x dépasse 20, alors la contrainte z de coût maintenance s'applique, sinon, elle ne s'applique pas.

Si un objet X1(variable de décision) dépasse 20kgs, alors le container Z n'est pas utilisé, c'est le container Y qui est utilisé.

Si un élément chimique X1 (variable de décision) dépasse un seuil de 20grammes, alors, une pénalité lui est appliquée, ce qui privilégie l'élement chimique X2 dans l'optimisation finale.

Déclencher une machine X2 si la production sur la machine X1 est > 1000, sinon, ne pas la déclencher (Mode ON/OFF)



## 1/ Si la production de x dépasse 20, alors la contrainte z de coût maintenance s'applique, sinon, elle ne s'applique pas.

On s'entraine sur un programme linéaire simple .
La création de la variable binaire z se passe en deux temps
- 1. On instancie la variable binaire
- 2. On crée une contrainte qui mets en jeu cette variable binaire

In [22]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
  
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# -----------------------------------
# On définit nos constantes
#
# -----------------------------------

# On spécifie le cout de maintenance
cout_maintenance = 10

# On spécifie le seuil de déclenchement de la maintenance
declencheur_maintenance = 20

# -----------------------------------
# On définit nos variables de décision
#
# -----------------------------------
  
# On Crée les variables de décision du problème , x et y sont des objets que l'usine produit
x = p.LpVariable("x", lowBound = 0, cat='Integer')   # Create a variable x >= 0 
y = p.LpVariable("y", lowBound = 0, cat='Integer')   # Create a variable y >= 0 

# Comme on a besoin d'appliquer un cout de maintenance conditionnel, on a besoin 
# de définir une variable binaire qui va se déclencher si la production de x est supérieure
# à 20 objets !
z = p.LpVariable("z", lowBound=0, cat='Binary')

# -----------------------------------
# On définit la fonction objectif
#
# -----------------------------------
  
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
# Ici, x est vendu 10.5 euros et y 8.5 euros, le coût de 10 euros de maintenance
#  est soustrait seulement si z est positive.
Lp_prob +=  10.5 * x + 8.5 * y - cout_maintenance * z


# -----------------------------------
# On définit nos contraintes
#
# ----------------------------------- 

# Heures de travail au mois
# Ca prends 3 heures de crée un objet x, et 2 heures de créer un objet z
Lp_prob += 3 * x + 2 * y  <= 420

# Il faut produire au minimum ce nombre d'éléments  :
Lp_prob += x  >= 100
Lp_prob += y  >= 40

# La contrainte binaire qu'on doit faire (pas encore faite):
# si x > 20 alors z = 1
Lp_prob += z  == 1

# -----------------------------------
# On résouds avec le solveur
#
# -----------------------------------

# Afficher le problème linéaire
# print(Lp_prob) 
status = Lp_prob.solve()   # Exécuter le solver
# print(p.LpStatus[status])   # Le statut de la solution

# Afficher la solution :
print(p.value(x),"Objets produits x")
print(p.value(y) , "Objets produits y"  )
print(p.value(z) , "La valeur de la variable binaire utilisée ou pas , 0 ou 1"  )
print(p.value(Lp_prob.objective) ,"est le nombre total d'objets produits" )


100.0 Objets produits x
60.0 Objets produits y
1.0 La valeur de la variable binaire utilisée ou pas , 0 ou 1
1550.0 est le nombre total d'objets produits
